**Programador:** Marco Gutierrez. Si se tiene alguna consulta, mandarla al siguiente [email](mailto:mgutierrezc@up.edu.pe)

# Obteniendo la información para el Scrapping

Importamos las librerias para el scrapping

In [12]:
import pandas as pd
import string
import WazeRouteCalculator
import logging
import numpy as np
import os
import re

## Importando las provincias/distritos a analizar

Primero, leeremos un archivo que contiene los elementos de los ubigeos del INEI. Se puede descargar entrando [aquí](http://webinei.inei.gob.pe:8080/sisconcode/proyecto/index.htm?proyectoTitulo=UBIGEO&proyectoId=3) y dando click a la opción **Excel** en la pestaña *Busqueda por Ubicación Geográfica*

In [13]:
ubigeos = pd.read_excel('ubigeo_inei.xls',skiprows=1)
ubigeos.head()

,Unnamed: 0,DEPARTAMENTO,Unnamed: 2,Unnamed: 3,PROVINCIA,DISTRITO
0,NaN,01 Amazonas,NaN,NaN,,
1,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,
2,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,01 Chachapoyas
3,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,02 Asunción
4,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,03 Balsas


Necesitamos las provincias y distritos de esta base, así que limpiaremos los valores de estas variables

In [14]:
ubigeos["DISTRITO"] = ubigeos["DISTRITO"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos["PROVINCIA"] = ubigeos["PROVINCIA"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos["DEPARTAMENTO"] = ubigeos["DEPARTAMENTO"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos.head()

,Unnamed: 0,DEPARTAMENTO,Unnamed: 2,Unnamed: 3,PROVINCIA,DISTRITO
0,NaN,Amazonas,NaN,NaN,,
1,NaN,Amazonas,NaN,NaN,Chachapoyas,
2,NaN,Amazonas,NaN,NaN,Chachapoyas,Chachapoyas
3,NaN,Amazonas,NaN,NaN,Chachapoyas,Asunción
4,NaN,Amazonas,NaN,NaN,Chachapoyas,Balsas


Limpiando las listas

Necesitaremos estas dos columnas como listas, pues después las usaremos para crear un DataFrame donde guardaremos los resultados del scrapping

In [15]:
# Creando las listas
provincias = list(ubigeos['PROVINCIA'])
distritos = list(ubigeos['DISTRITO'] + ", " + ubigeos['PROVINCIA'] + ", " + ubigeos['DEPARTAMENTO'])
for_removal_dist = list(", " + ubigeos['PROVINCIA'] + ", " + ubigeos['DEPARTAMENTO'])

In [16]:
# Eliminando duplicados
provincias = set(provincias)
distritos = set(distritos)
for_removal_dist = set(for_removal_dist)

# Eliminando algunos elementos inutiles ('', nan)
provincias = list(filter(lambda x: str(x) != 'nan', provincias))
distritos = list(filter(lambda x: str(x) != 'nan', distritos))

provincias.remove('')
provincias.remove('PROVINCIA')
distritos = [distr for distr in distritos if distr not in for_removal_dist]
distritos.remove('DISTRITO, PROVINCIA, DEPARTAMENTO')

Eliminando las tildes

In [17]:
# Definimos una función para eliminar las tildes
def normalize(s):
    replacements = (
        ("á", "a"),
        ("à", "a"),
        ("é", "e"),
        ("è", "e"),
        ("í", "i"),
        ("ì", "i"),
        ("ó", "o"),
        ("ò", "o"),
        ("ú", "u"),
        ("ù", "u")
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [18]:
# La aplicamos a cada elemento de nuestras provincias/distritos
nro_provs = len(provincias)
for i in range(nro_provs):
    provincias[i] = normalize(provincias[i])
    # De paso, les quitaremos espacios
    provincias[i] = provincias[i].strip()

nro_distrs = len(distritos)
for i in range(nro_distrs):
    distritos[i] = normalize(distritos[i])
    distritos[i] = distritos[i].split(",")
    
    # Eliminando los espacios dentro de los elementos de un distrito único (distr, prov, dept)
    item_distr = len(distritos[i])
    empty_distr = []
    for j in range(item_distr):
        distritos[i][j] = distritos[i][j].strip()
        empty_distr.append(distritos[i][j])
        
    # Joining our separated districts
    distritos[i] = ','.join(empty_distr)

# Scrapping de Distancias y Tiempos de viaje

Para realizar este scrapping, se tomará como referencia este [paquete](https://github.com/kovacsbalu/WazeRouteCalculator)

## Matrices base a nivel de distritos

Crearemos nuestra matriz de la siguiente manera

In [19]:
first_col = ["Origin Province\Destination Province"]
dist_distances = pd.DataFrame({"Origin Province\Destination Province": distritos}, 
                        columns = first_col+distritos)
dist_times = pd.DataFrame({"Origin Province\Destination Province": distritos}, 
                        columns = first_col+distritos)
dist_distances

,Origin Province\Destination Province,"Churuja,Bongara,Amazonas","Tingo,Luya,Amazonas","Pebas,Mariscal Ramon Castilla,Loreto","Tambo,La Mar,Ayacucho","San Cristobal,Picota,San Martin","Namballe,San Ignacio,Cajamarca","San Isidro,Huaytara,Huancavelica","Pulan,Santa Cruz,Cajamarca","San Sebastian,Cusco,Cusco",...,"Fernando Lores,Maynas,Loreto","Quishuar,Tayacaja,Huancavelica","Paranday,Otuzco,La Libertad","Yarumayo,Huanuco,Huanuco","Izcuchaca,Huancavelica,Huancavelica","San Mateo,Huarochiri,Lima","Pallanchacra,Pasco,Pasco","Carhuapampa,Ocros,Ancash","Pucala,Chiclayo,Lambayeque","La Merced,Churcampa,Huancavelica"
0,"Churuja,Bongara,Amazonas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Tingo,Luya,Amazonas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Pebas,Mariscal Ramon Castilla,Loreto",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Tambo,La Mar,Ayacucho",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"San Cristobal,Picota,San Martin",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,"San Mateo,Huarochiri,Lima",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1870,"Pallanchacra,Pasco,Pasco",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871,"Carhuapampa,Ocros,Ancash",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1872,"Pucala,Chiclayo,Lambayeque",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creando listas con los origenes/destinos para reemplazar en el dataframe

In [20]:
# Como queremos iterar sobre los nombres de las provincias
origen_destino_dist=[]

for dist_row in distritos:
    origen_destino_dist.append(dist_row+","+"Peru")

origen_destino_dist

['Churuja,Bongara,Amazonas,Peru',
 'Tingo,Luya,Amazonas,Peru',
 'Pebas,Mariscal Ramon Castilla,Loreto,Peru',
 'Tambo,La Mar,Ayacucho,Peru',
 'San Cristobal,Picota,San Martin,Peru',
 'Namballe,San Ignacio,Cajamarca,Peru',
 'San Isidro,Huaytara,Huancavelica,Peru',
 'Pulan,Santa Cruz,Cajamarca,Peru',
 'San Sebastian,Cusco,Cusco,Peru',
 'Chiclayo,Chiclayo,Lambayeque,Peru',
 'Lares,Calca,Cusco,Peru',
 'Megantoni,La Convencion,Cusco,Peru',
 'Aucallama,Huaral,Lima,Peru',
 'Polobaya,Arequipa,Arequipa,Peru',
 'Chocope,Ascope,La Libertad,Peru',
 'Catac,Recuay,Ancash,Peru',
 'Vizcatan del Ene,Satipo,Junin,Peru',
 'Ccochaccasa,Angaraes,Huancavelica,Peru',
 'Pallpata,Espinar,Cusco,Peru',
 'Lamay,Calca,Cusco,Peru',
 'Patapo,Chiclayo,Lambayeque,Peru',
 'Chota,Chota,Cajamarca,Peru',
 'El Tambo,Huancayo,Junin,Peru',
 'Pamparomas,Huaylas,Ancash,Peru',
 'Chongoyape,Chiclayo,Lambayeque,Peru',
 'Illimo,Lambayeque,Lambayeque,Peru',
 'Jose Crespo y Castillo,Leoncio Prado,Huanuco,Peru',
 'Capillas,Castrovirre

## ¿Cómo usar el paquete `WazeRouteCalculator`?

Ahora que ya tenemos todos los datos listos, nos falta activar la clase `WazeRouteCalculator` para que podamos extraer tiempos y distancias de waze. Ello se puede hacer así:

Obtenemos el logger de dicho paquete

In [21]:
logger = logging.getLogger('WazeRouteCalculator.WazeRouteCalculator')
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
logger.addHandler(handler)

## Usando `WazeRouteCalculator` para crear una función que calcule distancias/tiempos

In [28]:
def matrix_blocks(first_row, first_column, final_row_plus_one, final_col, df_distances, df_times, origin_dest):
    """
    Función para completar la matriz por bloques que comiencen desde "first_row" y "first_column" hasta "iterations_row" 
    y "iterations_column". Se creó porque ejecutar el sgte codigo para todas las celdas toma mucho tiempo en ejecutarse.
    
    Para hacer más eficiente la tarea, se puede ejecutar esto por partes hasta completarlo. Además, permite convertir la tarea
    en una grupal. Un equipo puede hacer que cada uno de sus miembros complete ciertos bloques (e.g. linea 1 hasta 12, otro 
    linea 12 hasta 24, y así sucesivamente)
    """
    # Codigo base para cada fila (comenzando con la 0)
    for row in range(first_row, final_row_plus_one):
        print(row)
        for col in range(first_column, final_col):
            # Definiendo un índice para nuestra lista con los origenes/destinos en función a la columna del dataframe
            # en la que queremos el resultado
            index_destination = col-1

            # Si las filas y columnas son consecutivas, las distancias y tiempo son de un sitio al mismo sitio, asi que 
            # debemos controlar ese factir
            if col-1==row:
                df_distances.iloc[row,col] = 0
                df_times.iloc[row,col] = 0

            else:
                # Obteniendo las distancias/tiempos correspondientes (el default es manejando un auto privado): 
                route = WazeRouteCalculator.WazeRouteCalculator(origin_dest[row], origin_dest[index_destination])
                # Como estamos en cuarentena (11-04-2020), no debemos colocar tiempo real, pues al no haber las restricciones
                # normales de viaje, habrá un sesgo negativo en el tiempo
                try:
                    route_results = route.calc_route_info(real_time=False)
                except:
                    route_results = [0, 0]
                # Usaremos diccionarios en lugar de variables simples pues permiten usar la func '.get'
                # La distancia está en km y la duración en minutos
                distance_duration = {'distance': route_results[1], 'duration': route_results[0]}
                
                # No siempre es posible viajar de un lugar a otro por restricciones geográficas, así que debemos definir qué ocurre
                # en casos así
                if distance_duration.get('distance'):
                    df_distances.iloc[row,col] = round(distance_duration['distance'], 2)
                else:
                    df_distances.iloc[row,col] = 0

                if distance_duration.get('duration'):
                    df_times.iloc[row,col] = round(distance_duration['duration'], 2)
                else:
                    df_times.iloc[row,col] = 0            

                # Para debuggear:
                # print('Origen: '+origen_destino[row]+' | Destino: '+origen_destino[index_destination]) 
                # print(prov_distances.iloc[row,col])

### Aplicando la función a las Matrices de Distritos

In [29]:
origen_destino_dist[0]
# origen_destino_dist[1]

'Churuja,Bongara,Amazonas,Peru'

In [32]:
primera_fila = 1
primera_col = 1
ult_fila_mas_uno = 1875
ult_col = len(distritos)+1
# ult_col = 4

matrix_blocks(primera_fila, primera_col, ult_fila_mas_uno, ult_col, dist_distances, dist_times, origen_destino_dist)

From: Tingo,Luya,Amazonas,Peru - to: Churuja,Bongara,Amazonas,Peru


1


Start coords: (-9.293227195739746, -75.99496459960938)
End coords: (-12.044655799865723, -77.02371215820312)
Time 681.88 minutes, distance 535.16 km.
From: Tingo,Luya,Amazonas,Peru - to: Pebas,Mariscal Ramon Castilla,Loreto,Peru
Start coords: (-9.293227195739746, -75.99496459960938)
End coords: (-12.056605339050293, -76.96033477783203)
Time 683.77 minutes, distance 528.42 km.
From: Pebas,Mariscal Ramon Castilla,Loreto,Peru - to: Churuja,Bongara,Amazonas,Peru


2


Start coords: (-12.056605339050293, -76.96033477783203)
End coords: (-12.044655799865723, -77.02371215820312)
Time 19.92 minutes, distance 12.59 km.
From: Pebas,Mariscal Ramon Castilla,Loreto,Peru - to: Tingo,Luya,Amazonas,Peru
Start coords: (-12.056605339050293, -76.96033477783203)
End coords: (-9.293227195739746, -75.99496459960938)
Time 698.57 minutes, distance 535.29 km.
From: Tambo,La Mar,Ayacucho,Peru - to: Churuja,Bongara,Amazonas,Peru


3


Start coords: (-12.948101997375488, -74.02031707763672)
End coords: (-12.044655799865723, -77.02371215820312)
Time 583.35 minutes, distance 651.79 km.
From: Tambo,La Mar,Ayacucho,Peru - to: Tingo,Luya,Amazonas,Peru
Start coords: (-12.948101997375488, -74.02031707763672)
End coords: (-9.293227195739746, -75.99496459960938)
Time 871.98 minutes, distance 778.13 km.
From: Tambo,La Mar,Ayacucho,Peru - to: Pebas,Mariscal Ramon Castilla,Loreto,Peru
Start coords: (-12.948101997375488, -74.02031707763672)
End coords: (-12.056605339050293, -76.96033477783203)
Time 577.68 minutes, distance 646.08 km.


In [33]:
dist_distances.head()

,Origin Province\Destination Province,"Churuja,Bongara,Amazonas","Tingo,Luya,Amazonas","Pebas,Mariscal Ramon Castilla,Loreto","Tambo,La Mar,Ayacucho","San Cristobal,Picota,San Martin","Namballe,San Ignacio,Cajamarca","San Isidro,Huaytara,Huancavelica","Pulan,Santa Cruz,Cajamarca","San Sebastian,Cusco,Cusco",...,"Fernando Lores,Maynas,Loreto","Quishuar,Tayacaja,Huancavelica","Paranday,Otuzco,La Libertad","Yarumayo,Huanuco,Huanuco","Izcuchaca,Huancavelica,Huancavelica","San Mateo,Huarochiri,Lima","Pallanchacra,Pasco,Pasco","Carhuapampa,Ocros,Ancash","Pucala,Chiclayo,Lambayeque","La Merced,Churcampa,Huancavelica"
0,"Churuja,Bongara,Amazonas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Tingo,Luya,Amazonas",535.16,0,528.42,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Pebas,Mariscal Ramon Castilla,Loreto",12.59,535.29,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Tambo,La Mar,Ayacucho",651.79,778.13,646.08,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"San Cristobal,Picota,San Martin",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
